In [37]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split, cross_val_score

from utils.load_data import alldat, pre_process, comp_avg_signal
from utils.algo import sliding_window

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
nonfilter_overt = pre_process(subj=0, exp=0, hpf=50)
nonfilter_imag = pre_process(subj=0, exp=1, hpf=50)
sample_overt = comp_avg_signal(nonfilter_overt, 9960, 'hand')
sample_imag = comp_avg_signal(nonfilter_imag, 9960, 'hand')
signals = {'overt': sample_overt, 'imag': sample_imag}

In [34]:
def prepare_data(data, cv, **window_kwargs):
    """
    data: dictionary of shape: 
            {'overt': overt_data, 'imag': imag_data}
    cv: % test data
    labels: 0 is overt, 1 imag
    """
    _label_map = {'overt':0,'imag':1}
    all_features = []
    all_labels = []
    for chan in range(46):
        for k in signals.keys():
            windows = sliding_window(signals[k][:,chan], **window_kwargs)
            [all_features.append(w) for w in windows]
            [all_labels.append(_label_map[k]) for i in range(len(windows))]
    if cv:
        X_train, X_test, y_train, y_test = train_test_split(all_features, all_labels, test_size=cv)
        return X_train, X_test, y_train, y_test
    else:
        return all_features, all_labels

X_train, X_test, y_train, y_test =  prepare_data(signals, 0.1, win=int(9960/10))
X, y =  prepare_data(signals, None, win=int(9960/10))

In [39]:
clf = svm.SVC()
# clf.fit(X_train, y_train)
scores = cross_val_score(clf, X, y, cv=5, scoring='recall_macro')
scores.mean(0)

0.6643402879811932

In [7]:
sample_overt.shape

(9960, 46)

In [20]:
x = [[1,2,3],[1,2,3]]
y = []
[y.append(i) for i in x]
y

[[1, 2, 3], [1, 2, 3]]